In [1]:
import torch
import gym
import torch
import gym
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
from torch.distributions import Normal
import scipy.signal
eps = np.finfo(np.float32).eps.item()
#torch.manual_seed(100)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [2]:
def discount_cumsum(x, discount):
    """
    magic from rllab for computing discounted cumulative sums of vectors.
    input: 
        vector x, 
        [x0, 
         x1, 
         x2]
    output:
        [x0 + discount * x1 + discount^2 * x2,  
         x1 + discount * x2,
         x2]
    """
    return scipy.signal.lfilter([1], [1, float(-discount)], x[::-1], axis=0)[::-1]

In [3]:
class ValueNetwork(nn.Module):
    def __init__(self, obs_dimension, sizes, act = nn.ReLU):
        super(ValueNetwork, self).__init__()
        sizes = [obs_dimension] + sizes + [1]
        out_activation = nn.Identity
        self.layers = []
        for j in range(0,len(sizes) - 1):
            act_l = act if j < len(sizes) -2 else out_activation
            self.layers+=[nn.Linear(sizes[j], sizes[j+1]), act_l()]
        self.v = nn.Sequential(*self.layers)
    def forward(self, x):
        return self.v(x)

In [4]:
class PolicyNetworkCat(nn.Module):
    def __init__(self, obs_dimension, sizes, action_dimension, act= nn.ReLU):
        super(PolicyNetworkCat, self).__init__()
        sizes = [obs_dimension] + sizes + [action_dimension]
        out_activation = nn.Identity
        self.layers = []
        for j in range(0,len(sizes) - 1):
            act_l = act if j < len(sizes) -2 else out_activation
            self.layers+=[nn.Linear(sizes[j], sizes[j+1]), act_l()]
        self.pi = nn.Sequential(*self.layers)
    def forward(self, x):
        score = self.pi(x)
        probs = F.softmax(score,dim = 1)
        dist = torch.distributions.Categorical(probs=probs)
        return dist

In [5]:
class PolicyNetworkGauss(nn.Module):
    def __init__(self, obs_dimension, sizes, action_dimension):
        super(PolicyNetworkGauss, self).__init__()
        sizes = [obs_dimension] + sizes + [action_dimension]
        act = nn.Tanh
        out_activation = nn.Identity
        self.layers = []
        for j in range(0,len(sizes) - 1):
            act_l = act if j < len(sizes) -2 else out_activation
            self.layers+=[nn.Linear(sizes[j], sizes[j+1]), act_l()]
        self.mu = nn.Sequential(*self.layers)
        log_std = -0.5*np.ones(action_dimension, dtype=np.float32)
        self.log_std = torch.nn.Parameter(torch.as_tensor(log_std))
    def forward(self, x):
        mean = self.mu(x)
        std = torch.exp(self.log_std)
        dist = Normal(mean, std)
        return dist

In [6]:
env = gym.make('CartPole-v1')
sizes = [128]
obs_dimension = env.observation_space.shape
action_dimension = env.action_space.shape
print(obs_dimension)
print(env.action_space.n)
v = ValueNetwork(*obs_dimension, sizes)
pi = PolicyNetworkGauss(*obs_dimension, sizes, 2)

(4,)
2


In [7]:
class PPOBuffer:
    """
    A buffer for storing trajectories experienced by a PPO agent interacting
    with the environment, and using Generalized Advantage Estimation (GAE-Lambda)
    for calculating the advantages of state-action pairs.
    """

    def __init__(self, obs_dim, act_dim, size, gamma=0.99, lam=0.95):
        self.obs_buf = np.zeros((size, obs_dim), dtype=np.float32)
        self.act_buf = np.zeros((size,), dtype=np.float32)
        self.adv_buf = np.zeros(size, dtype=np.float32)
        self.rew_buf = np.zeros(size, dtype=np.float32)
        self.ret_buf = np.zeros(size, dtype=np.float32)
        self.val_buf = np.zeros(size, dtype=np.float32)
        self.logp_buf = np.zeros(size, dtype=np.float32)
        self.gamma, self.lam = gamma, lam
        self.ptr, self.path_start_idx, self.max_size = 0, 0, size

    def store(self, obs, act, rew, val, logp):
        """
        Append one timestep of agent-environment interaction to the buffer.
        """
        assert self.ptr < self.max_size     # buffer has to have room so you can store
        self.obs_buf[self.ptr] = obs
        self.act_buf[self.ptr] = act
        self.rew_buf[self.ptr] = rew
        self.val_buf[self.ptr] = val
        self.logp_buf[self.ptr] = logp
        self.ptr += 1

    def finish_path(self, last_val=0):
        """
        Call this at the end of a trajectory, or when one gets cut off
        by an epoch ending. This looks back in the buffer to where the
        trajectory started, and uses rewards and value estimates from
        the whole trajectory to compute advantage estimates with GAE-Lambda,
        as well as compute the rewards-to-go for each state, to use as
        the targets for the value function.

        The "last_val" argument should be 0 if the trajectory ended
        because the agent reached a terminal state (died), and otherwise
        should be V(s_T), the value function estimated for the last state.
        This allows us to bootstrap the reward-to-go calculation to account
        for timesteps beyond the arbitrary episode horizon (or epoch cutoff).
        """

        path_slice = slice(self.path_start_idx, self.ptr)
        rews = np.append(self.rew_buf[path_slice], last_val)
        vals = np.append(self.val_buf[path_slice], last_val)
       # print(rews)
       # print(vals)
        # the next two lines implement GAE-Lambda advantage calculation
        deltas = rews[:-1] + self.gamma * vals[1:] - vals[:-1]
        self.adv_buf[path_slice] = discount_cumsum(deltas, self.gamma * self.lam)
        
        # the next line computes rewards-to-go, to be targets for the value function
        self.ret_buf[path_slice] = discount_cumsum(rews, self.gamma)[:-1]
        
        self.path_start_idx = self.ptr

    def get(self):
        """
        Call this at the end of an epoch to get all of the data from
        the buffer, with advantages appropriately normalized (shifted to have
        mean zero and std one). Also, resets some pointers in the buffer.
        """
        assert self.ptr == self.max_size    # buffer has to be full before you can get
        self.ptr, self.path_start_idx = 0, 0
        # the next two lines implement the advantage normalization trick

        self.adv_buf = (self.adv_buf - self.adv_buf.mean()) / self.adv_buf.std()
        data = dict(obs=self.obs_buf, act=self.act_buf, ret=self.ret_buf,
                    adv=self.adv_buf, logp=self.logp_buf)
        return {k: torch.as_tensor(v, dtype=torch.float32) for k,v in data.items()}


In [8]:
def ppo(env, seed = 0, buff_size = 4000, train_time_steps = 1000000, gamma = 0.99, clip_ratio = 0.2, lr_pi = 3e-4, 
        lr_vf = 1e-3, pi_train_itrs = 80, v_train_itrs = 80, lam = 0.97, max_ep_len = 500):
        obs_dim = env.observation_space.shape
        action_dim = 2
        h_sizes = [64]
        vi = ValueNetwork(*obs_dim, h_sizes).to(device)
        pi = PolicyNetworkCat(*obs_dim, h_sizes, action_dim).to(device)
        data_buff = PPOBuffer(*obs_dim, 1, buff_size)
        policy_opt = optim.Adam(pi.parameters(), lr = lr_pi)
        value_opt = optim.Adam(vi.parameters(), lr = lr_vf)
        obs = env.reset()
        curr_time_step = 0
        #pbar = tqdm(total = train_time_steps)
        num_episode = 0
        ep_rewards = [0]
        while curr_time_step < train_time_steps: 
                for t in range(0, buff_size):
                        obs_tensor = torch.from_numpy(obs).float().unsqueeze(0).to(device)
                        with torch.no_grad():
                                m = pi(obs_tensor)
                                a = m.sample()
                       # print(a.detach().numpy())
                       # print(m)
                        #return
                                logp = m.log_prob(a)
                                obs_new, rew, done, _ = env.step(a.item())
                                ep_rewards[num_episode]+=rew 
                                v = vi(obs_tensor)
                                obs = obs_new
                        data_buff.store(obs, a.numpy(), rew, v.numpy(), logp.numpy())
                        if done or t == buff_size-1:
                                if done:
                                        v_ = 0.
                                       # print('done')
                                        obs = env.reset()
                                        done = False
                                        num_episode+=1
                                        ep_rewards.append(0)
                                        if num_episode %100 == 0:
                                                print(f'episode: {num_episode-1} \t episode_reward: {np.mean(ep_rewards[-10:-2])} \t total steps:{curr_time_step}')
                                else:
                                        v_ = vi(torch.from_numpy(obs).float().unsqueeze(0))
                                        v_ = v_.detach().numpy()
                                data_buff.finish_path(v_)
                        curr_time_step+=1
                       # pbar.update(1)
                data = data_buff.get()
                for j in range(0, pi_train_itrs):
                        policy_opt.zero_grad()
                        act, adv, o, logp_old= data['act'], data['adv'], data['obs'], data['logp']
                        act_dist = pi(o)
                        logp = act_dist.log_prob(act)
                        ratio = torch.exp(logp - logp_old)
                       # print(ratio)
                        clip_adv = torch.clamp(ratio, 1-clip_ratio, 1+clip_ratio) * adv
                        loss_pi = -(torch.min(ratio * adv, clip_adv)).mean()
                        print(f'loss pi: {loss_pi}')
                        print(f'adv: {adv}')
                       # print("action")
                       # print(act)
                       # print("dist")
                       # print(act_dist)
                        #policy_loss = -(logp*adv).mean()
                        #policy_loss.backward()
                        loss_pi.backward()
                        policy_opt.step()
                for i in range (0, v_train_itrs):
                        value_opt.zero_grad()
                        ret, ob = data['ret'], data['obs']
                        val = vi(ob)
                        value_loss = F.mse_loss(val, ret)
                       # print(f'value_loss: {value_loss}')
                        value_loss.backward()
                        value_opt.step()
               # print(curr_time_step)
                #pbar.update(1)
       # pbar.close()



In [9]:
ppo(env)

786,  ...,  0.5518,  0.3315,  0.0298])
loss pi: -0.11435288935899734
adv: tensor([-0.6053, -1.0263, -1.4786,  ...,  0.5518,  0.3315,  0.0298])
loss pi: -0.114400215446949
adv: tensor([-0.6053, -1.0263, -1.4786,  ...,  0.5518,  0.3315,  0.0298])
loss pi: -0.11443979293107986
adv: tensor([-0.6053, -1.0263, -1.4786,  ...,  0.5518,  0.3315,  0.0298])
loss pi: -0.11448216438293457
adv: tensor([-0.6053, -1.0263, -1.4786,  ...,  0.5518,  0.3315,  0.0298])
loss pi: -0.11453283578157425
adv: tensor([-0.6053, -1.0263, -1.4786,  ...,  0.5518,  0.3315,  0.0298])
loss pi: -0.11459033936262131
adv: tensor([-0.6053, -1.0263, -1.4786,  ...,  0.5518,  0.3315,  0.0298])
loss pi: -0.11465239524841309
adv: tensor([-0.6053, -1.0263, -1.4786,  ...,  0.5518,  0.3315,  0.0298])
loss pi: -0.11471326649188995
adv: tensor([-0.6053, -1.0263, -1.4786,  ...,  0.5518,  0.3315,  0.0298])
loss pi: -0.1147678941488266
adv: tensor([-0.6053, -1.0263, -1.4786,  ...,  0.5518,  0.3315,  0.0298])
loss pi: -0.1148157641291618

KeyboardInterrupt: 

In [ ]:
v.__dict__

In [ ]:
def combined_shape(length, shape=None):
    if shape is None:
        return (length,)
    return (length, shape) if np.isscalar(shape) else (length, *shape)

In [ ]:
shape = env.action_space.shape

In [ ]:
*shape

In [ ]:
combined_shape(100 ,shape)

In [ ]:
np.isscalar(shape)

In [ ]:
(100, *shape)